# **Импорт библиотек**

In [1]:
from pyspark.sql import SparkSession, SQLContext
import pyspark.sql.functions as F
import logging
import os
from airflow.models import Variable
from dotenv import load_dotenv
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DataType
import requests
import json
import boto3
import minio
import mimesis
from mimesis import Person, Generic, Address, Finance, Datetime, Choice
from mimesis.locales import Locale
from mimesis import Code
from mimesis.enums import TimestampFormat
import datetime
import logging

# **Create SparkSession**

In [2]:
# Настройка логирования
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# Загрузка и проверка переменных окружения
load_dotenv()

try:
    # Создание SparkSession
    spark = SparkSession.builder \
        .appName("MinIO Data Reader") \
        .config("spark.hadoop.fs.s3a.access.key", os.getenv("MINIO_ACCESS_KEY")) \
        .config("spark.hadoop.fs.s3a.secret.key", os.getenv("MINIO_SECRET_KEY")) \
        .config("spark.hadoop.fs.s3a.endpoint", os.getenv("MINIO_ENDPOINT")) \
        .config("spark.hadoop.fs.s3a.path.style.access", "true") \
        .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
        .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
        .config("spark.jars", "/home/jovyan/jars/hadoop-aws-3.3.1.jar,/home/jovyan/jars/aws-java-sdk-bundle-1.11.901.jar,/home/jovyan/jars/iceberg-spark-runtime-3.5_2.12-1.5.0.jar") \
        .config("spark.sql.catalog.nessie", "org.apache.iceberg.spark.SparkCatalog") \
        .config("spark.sql.catalog.nessie.type", "nessie") \
        .config("spark.sql.catalog.nessie.uri", "http://nessie-server:19120/api/v2") \
        .config("spark.sql.catalog.nessie.ref", "main") \
        .config("spark.sql.catalog.nessie.warehouse", "s3a://datalake/warehouse") \
        .config("spark.sql.catalog.nessie.io-impl", "org.apache.iceberg.hadoop.HadoopFileIO") \
        .getOrCreate()

    
    logging.info('SparkSession успешно создана для работы с MinIO')
except Exception as e:
    logging.error(f'SparkSession не создана по причине: {e}')


[2025-09-19T21:04:04.157+0000] {176353067.py:31} INFO - SparkSession успешно создана для работы с MinIO


# **Проверяем каталог Nessie**

In [ ]:
spark.sql("SHOW NAMESPACES IN nessie").show()

# **Создаем namespace в каталоге Nessie**

In [ ]:
spark.sql("CREATE NAMESPACE IF NOT EXISTS nessie.dev").show()

In [ ]:
path = "transactions.csv"
schema = StructType(fields=[
        StructField("Customer ID", StringType()),
        StructField("Transaction Date", StringType()),
        StructField("Product Purchased", StringType()),
        StructField("Transaction Amount", DoubleType()),
        StructField("Payment Method", StringType()),
        StructField("Transaction Status", StringType()),
        StructField("Transaction Type", StringType())
])
transactions_df = spark.read.csv(path, schema=schema, sep=",", header=True)
transactions_df.show(10)

In [ ]:
transactions_df.printSchema()

# **Запись в S3**

In [ ]:
output_path = "s3a://datalake/transactions.parquet"
transactions_df.write.mode("overwrite").parquet(output_path)

# **Проверка файлов в бакете**

In [ ]:
raw_endpoint = os.getenv("MINIO_ENDPOINT")

if not raw_endpoint.startswith(('http://', 'https://')):
    endpoint_url = 'http://' + raw_endpoint
else:
    endpoint_url = raw_endpoint
    

s3_creds = {
    "aws_access_key_id": os.getenv("MINIO_ACCESS_KEY"),
    "aws_secret_access_key": os.getenv("MINIO_SECRET_KEY"),
    "endpoint_url": endpoint_url
}

s3 = boto3.client("s3", **s3_creds)

my_bucket = "datalake"

response = s3.list_objects_v2(Bucket=my_bucket)

for obj in response.get('Contents', []):
    print(obj['Key'])

# **Чтение из S3**

In [3]:
s3_path = "s3a://datalake/transactions.parquet/"
df_transactions = spark.read.parquet(s3_path)
df_transactions.show(10)

+--------------------+----------------+-----------------+------------------+--------------+------------------+----------------+
|         Customer ID|Transaction Date|Product Purchased|Transaction Amount|Payment Method|Transaction Status|Transaction Type|
+--------------------+----------------+-----------------+------------------+--------------+------------------+----------------+
|7810b819-f86d-498...|      2025-03-05|          quickly|417.52795241907484|          Cash|         Completed|        In-Store|
|c1426052-e306-459...|      2025-03-19|         material|174.45190084753798|        PayPal|            Failed|          Online|
|99a40289-d53d-4e2...|      2025-04-09|           travel|28.593285457686974|     Gift Card|            Failed|        In-Store|
|984e8efc-9a2f-437...|      2025-02-02|              cut|  78.3398992520693|        PayPal|            Failed|        In-Store|
|741d0bea-1f72-435...|      2025-04-13|              she| 288.2127996249924|        PayPal|           Pe

In [4]:
df_transactions.printSchema()

root
 |-- Customer ID: string (nullable = true)
 |-- Transaction Date: string (nullable = true)
 |-- Product Purchased: string (nullable = true)
 |-- Transaction Amount: double (nullable = true)
 |-- Payment Method: string (nullable = true)
 |-- Transaction Status: string (nullable = true)
 |-- Transaction Type: string (nullable = true)



# **Добавим новую колонку dt с данными из колонки Transaction Date и приведем ее к типу Date**

In [5]:
new_df_transactions = df_transactions.withColumn("dt", F.to_date("Transaction Date", "yyyy-MM-dd"))

In [ ]:
new_df_transactions.show(10)

In [6]:
new_df_transactions.printSchema()

root
 |-- Customer ID: string (nullable = true)
 |-- Transaction Date: string (nullable = true)
 |-- Product Purchased: string (nullable = true)
 |-- Transaction Amount: double (nullable = true)
 |-- Payment Method: string (nullable = true)
 |-- Transaction Status: string (nullable = true)
 |-- Transaction Type: string (nullable = true)
 |-- dt: date (nullable = true)



# **Создадим новый dataframe с корректными колонками**

In [7]:
df_project = new_df_transactions.withColumnRenamed("Customer ID", "customer_id") \
            .withColumnRenamed("Product Purchased", "product_purchased") \
            .withColumnRenamed("Transaction Amount", "amount") \
            .withColumnRenamed("Payment Method", "payment_method") \
            .withColumnRenamed("Transaction Status", "transaction_status") \
            .withColumnRenamed("Transaction Type", "transaction_type") \
            .select("customer_id", "product_purchased", "amount", "payment_method", "transaction_status", "transaction_type", "dt") 

In [13]:
df_project.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- product_purchased: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- payment_method: string (nullable = true)
 |-- transaction_status: string (nullable = true)
 |-- transaction_type: string (nullable = true)
 |-- dt: date (nullable = true)



# **Приведем колонку amount к типу Decimal(18, 2)**

**Деньги ≠ double. double — двоичная дробь, даёт «копеечные» погрешности (0.1+0.2≠0.3)**

In [14]:
df_project = df_project.withColumn(
                      "amount",
                      F.round(F.col("amount"), 2).cast("DECIMAL(18,2)")
                             )

In [15]:
df_project.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- product_purchased: string (nullable = true)
 |-- amount: decimal(18,2) (nullable = true)
 |-- payment_method: string (nullable = true)
 |-- transaction_status: string (nullable = true)
 |-- transaction_type: string (nullable = true)
 |-- dt: date (nullable = true)



In [16]:
df_project.count()

100000

# **Проверка на NULL**

In [17]:
df_project.filter(new_df_transactions["dt"].isNotNull()) \
                   .count()

100000

In [18]:
df_project.filter(new_df_transactions["dt"].isNull()) \
                   .count()

0

# **MIN/MAX по дате**

In [19]:
df_project.agg(F.min("dt")) \
                   .show()

+----------+
|   min(dt)|
+----------+
|2025-01-01|
+----------+



In [20]:
df_project.agg(F.max("dt")) \
                   .show()

+----------+
|   max(dt)|
+----------+
|2025-05-18|
+----------+



In [23]:
df_project.agg(F.max("dt")) \
                   .show()

+----------+
|   max(dt)|
+----------+
|2025-05-18|
+----------+



# **Создание iceberg таблицы в каталоге Nessie**

In [34]:
spark.sql("""CREATE TABLE nessie.dev.transactions_sample (
                        customer_id VARCHAR(128), 
                        product_purchased VARCHAR(128), 
                        amount DECIMAL(18, 2), 
                        payment_method VARCHAR(128), 
                        transaction_status VARCHAR(128), 
                        transaction_type VARCHAR(128), 
                        dt DATE
                        )
            USING iceberg 
            PARTITIONED BY (months(dt));
""")

DataFrame[]

# **Вывести список таблиц из каталога**

In [38]:
spark.sql("SHOW TABLES IN nessie").show()

+---------+-------------------+-----------+
|namespace|          tableName|isTemporary|
+---------+-------------------+-----------+
|      dev|transactions_sample|      false|
+---------+-------------------+-----------+



# **Описание таблицы**

In [36]:
spark.sql("DESCRIBE TABLE nessie.dev.transactions_sample").show()

+------------------+-------------+-------+
|          col_name|    data_type|comment|
+------------------+-------------+-------+
|       customer_id|       string|   NULL|
| product_purchased|       string|   NULL|
|            amount|decimal(18,2)|   NULL|
|    payment_method|       string|   NULL|
|transaction_status|       string|   NULL|
|  transaction_type|       string|   NULL|
|                dt|         date|   NULL|
|                  |             |       |
|    # Partitioning|             |       |
|            Part 0|   months(dt)|       |
+------------------+-------------+-------+



# **Запрос на вывод 10 строк**

In [39]:
spark.sql("SELECT * FROM nessie.dev.transactions_sample LIMIT 10").show()

+--------------------+-----------------+------+--------------+------------------+----------------+----------+
|         customer_id|product_purchased|amount|payment_method|transaction_status|transaction_type|        dt|
+--------------------+-----------------+------+--------------+------------------+----------------+----------+
|99a40289-d53d-4e2...|           travel| 28.59|     Gift Card|            Failed|        In-Store|2025-04-09|
|741d0bea-1f72-435...|              she|288.21|        PayPal|           Pending|          Online|2025-04-13|
|dfb6d83a-db02-456...|           summer|161.91|     Gift Card|         Completed|          Online|2025-04-02|
|45850d77-4b5e-482...|       throughout| 49.26|     Gift Card|         Completed|        In-Store|2025-04-13|
|c028e4e4-8e02-4dc...|         position| 69.25|     Gift Card|         Completed|        In-Store|2025-04-24|
|3e4b0167-c762-4f2...|           really|164.42|     Gift Card|            Failed|        In-Store|2025-04-23|
|075f43b2-

# **Расширенное описание таблицы**

In [42]:
spark.sql("DESCRIBE TABLE EXTENDED nessie.dev.transactions_sample").show(truncate=False)

+----------------------------+------------------------------+-------+
|col_name                    |data_type                     |comment|
+----------------------------+------------------------------+-------+
|customer_id                 |string                        |NULL   |
|product_purchased           |string                        |NULL   |
|amount                      |decimal(18,2)                 |NULL   |
|payment_method              |string                        |NULL   |
|transaction_status          |string                        |NULL   |
|transaction_type            |string                        |NULL   |
|dt                          |date                          |NULL   |
|                            |                              |       |
|# Partitioning              |                              |       |
|Part 0                      |months(dt)                    |       |
|                            |                              |       |
|# Metadata Columns 

# **Вывод местонахождения файла в Minio**

In [43]:
spark.sql("""
  DESCRIBE TABLE EXTENDED nessie.dev.transactions_sample
""").where("col_name = 'Location'").show(truncate=False)

+--------+-------------------------------------------------------------------------------------+-------+
|col_name|data_type                                                                            |comment|
+--------+-------------------------------------------------------------------------------------+-------+
|Location|s3a://datalake/warehouse/dev/transactions_sample_321b4108-d636-454b-b042-43089ef68347|       |
+--------+-------------------------------------------------------------------------------------+-------+



In [45]:
spark.sql("SELECT file_path FROM nessie.dev.transactions_sample.files LIMIT 5").show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|file_path                                                                                                                                                                |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|s3a://datalake/warehouse/dev/transactions_sample_321b4108-d636-454b-b042-43089ef68347/data/dt_month=2025-04/00000-29-60365a04-4349-491d-a040-06b910f34491-0-00001.parquet|
|s3a://datalake/warehouse/dev/transactions_sample_321b4108-d636-454b-b042-43089ef68347/data/dt_month=2025-01/00001-30-60365a04-4349-491d-a040-06b910f34491-0-00001.parquet|
|s3a://datalake/warehouse/dev/transactions_sample_321b4108-d636-454b-b042-43089ef68347/data/dt_month=2025-05/00002-31-60365a04-4349-491d-a04

# **Удалить таблицу из каталога**

In [33]:
spark.sql("DROP TABLE nessie.dev.transactions_sample").show()

++
||
++
++



# **Проверка ветки каталога**

In [ ]:
spark.conf.get("spark.sql.catalog.nessie.ref", "UNSET")

# **Запись dataframe в Iceberg таблицу**

In [37]:
df_project.writeTo("nessie.dev.transactions_sample").append()

# **Проверить список Jars**

In [ ]:
print(spark.sparkContext._jsc.sc().listJars())

# **Узнать версию Hadoop-клиента**

In [ ]:
print(spark.sparkContext._jvm.org.apache.hadoop.util.VersionInfo.getVersion())